In [1]:
import sys

sys.path.append('../..')

In [2]:
import dill

loaded_data = None

with open('../simulations/data.pkl', 'rb') as f:
    loaded_data = dill.load(f)

keys = loaded_data.keys()
keys.sort()
print keys

['N', 'credit', 'randomization', 'underlyings']


In [3]:
# Instead of taking 365 standard days or 252 trading days
# in order to get some easy computations for the eqty and df time grids
# I chose to take 360 days of tradings

step = 1/360.
delta = 5*step

maturity = loaded_data["randomization"]["maturity"]

#Discount

In [4]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.02
discount = ConstantRateDiscountFactor(r)

# Underlying

In [5]:
udlyings = loaded_data["underlyings"]

bm = udlyings[0]
time_grid = bm.time

# Derivative

In [6]:
derivatives_nb = 1

In [7]:
from finance.products.european.assetswap import (
    SwapContract,
)

swap_delta = 0.5

swap_dates = SwapContract.generate_payment_dates(0, maturity, swap_delta)
swap = SwapContract(bm, discount, swap_dates)

price_0 = swap.price(0.)

print swap
print "\nPrice swap at t=0 = ",price_0

Swap contract of maturity T = 5 years, over S^0 with strike K = 104.560, paying at {0.00, 0.50, 1.00, 1.50, 2.00, 2.50, 3.00, 3.50, 4.00, 4.50, 5.00}

Price swap at t=0 =  -5.68434188608e-14


# Exposure

In [8]:
from risk.exposures import EuropeanQuantileGeomBrownianExposure

quantile_im = 0.80
prop_df = 0.2

hist_drift = bm.drifts[0][0]
hist_vol = bm.vols[0][0]

exposure = EuropeanQuantileGeomBrownianExposure(swap, discount, hist_drift, hist_vol)

# Indexes stuffs

## Ids of $B$ and $C$

In [9]:
copula = loaded_data["credit"]["copula"]

bc_subsets_indexes = loaded_data["credit"]["bc_subsets_indexes"]

# Bank id
b_id = 64

# Counterparty id
c_id = 17
c_ids = [c_id]

bc_ids = [b_id, c_id]

print "Bank id: %s (nb = 1) \n"%b_id
print "Counterparties id: %s (nb = 1)"%(c_id)

Bank id: 64 (nb = 1) 

Counterparties id: 17 (nb = 1)


In [10]:
obligors_nb = len(copula.subsets[bc_subsets_indexes[-1]][0])

print "Obligor numbers: %s"%obligors_nb

Obligor numbers: 125


## Subsets indexes of the copula for $B$ and $C$

In [11]:
b_subsets_indexes = copula.get_indexes_including(b_id)
c_subsets_indexes = copula.get_indexes_including(c_id)

c_subsets_indexes = list(set(c_subsets_indexes)-set(b_subsets_indexes))
    
b_subsets_indexes.sort()
c_subsets_indexes.sort()
    
print "Subsets that generated default times: %s\n"%bc_subsets_indexes
print "Bank subsets indexes: %s\n"%b_subsets_indexes
print "Counterparties subsets indexes: %s"%c_subsets_indexes

Subsets that generated default times: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129]

Bank subsets indexes: [64, 126, 127, 128, 129]

Counterparties subsets indexes: [17]


# Portfolio construction

In [12]:
import numpy as np

p_fixed = 100.
strike = swap.strike

delta_times = swap.delta_time
discount_factors = [discount(t) for t in swap.pillars[1:]]

delta_beta_sum = np.dot(delta_times, discount_factors)

notional = p_fixed / (strike*delta_beta_sum)

In [13]:
from finance.portfolio import CCPPortfolio

pos = CCPPortfolio.generate_1_vs_all_positions(b_id, bc_ids, obligors_nb)
positions = np.array(pos*notional).reshape(pos.size, 1)

In [14]:
port = CCPPortfolio(positions, [swap], [exposure])

# VM, IM and DF accounts

In [15]:
from ccp.accounts import Accounts, DFAccounts
from ccp.states import MembersState

states = MembersState(obligors_nb)
vm_accounts = Accounts(states, derivatives_nb)
im_accounts = Accounts(states, derivatives_nb)
df_accounts = DFAccounts(states, derivatives_nb)

# Skin in the game

In [16]:
from ccp.sig import SkinInTheGame

sig = SkinInTheGame(0.)

# Regulatory capital

In [17]:
from risk.basel.exposures import EuropeanBaselGeometricBrownianExposure

k_exposure = EuropeanBaselGeometricBrownianExposure(swap, hist_drift, hist_vol)

In [18]:
from risk.basel.capital import CCPRegulatoryCapital2014

regul_cap = CCPRegulatoryCapital2014(vm_accounts, im_accounts, df_accounts, sig, port, [k_exposure])

In [19]:
cap_risk_horizon = -1
cap_risk_period = 5./360
cap_conf_level_95 = 0.95
cap_conf_level_99 = 0.99

# Funding and capital parameters

In [20]:
lambda_ = 0.
k_ = 0.1
c_0 = -.0020

bank_recov = 0.4

# Creation of the different taks

In [21]:
class ITask(object):
    level = None
    
    def perform(self, **kwargs):
        raise NotImplementedError()

In [22]:
class PutMembersAtDefaultTask(ITask):
    level = 1
    
    def __init__(self, **kwargs):
        self.__states = kwargs["states"]
        self.potential_defaulters = kwargs["defaulters"]
        self.defaulters = []
        
    def perform(self, **kwargs):
        self.__tau = kwargs["t"]
        
        for i in self.potential_defaulters:
            for j in list(i):
                if self.__states.is_alive(j):
                    self.defaulters.append(j)
                    self.__states.die(j)
            
        self.defaulters.sort()

In [23]:
class LiquidatePortfolioTask(ITask):
    level = 1
    
    def __init__(self, **kwargs):
        self.__cm_index = kwargs["bank_index"]
        
        self.__default_task = kwargs["default_task"]
                
        self.__port = kwargs['portfolio']
        self.__vm_accounts = kwargs['vm_accounts']
        self.__im_accounts = kwargs['im_accounts']
        self.__df_accounts = kwargs['df_accounts']
                
        self.__discount = kwargs["discount"]
        self.__sig = kwargs['sig']
        
        self.results = {}
        
    def perform(self, **kwargs):
        self.__tau_delta = kwargs["t"]
        
        self.new_defaulters_ids = self.__default_task.defaulters
        
        contract_prices = [d.price(self.__tau_delta) for d in self.__port.derivatives]
        p_tau_delta = port.compute_value(contract_prices)[self.new_defaulters_ids, :]
        p_tau_delta = np.sum(p_tau_delta, axis=1)
        
        q_tau_delta = p_tau_delta
        
        gamma_tau_delta = self.__vm_accounts.amounts + self.__im_accounts.amounts + self.__df_accounts.amounts
        gamma_tau_delta = gamma_tau_delta[self.new_defaulters_ids, :]
        gamma_tau_delta = np.sum(gamma_tau_delta, axis=1)
        
        loss_tau_delta = q_tau_delta - gamma_tau_delta
        
        remain_collat_loss = 0.
        for l in loss_tau_delta:
            remain_collat_loss += np.maximum(l, 0)
            
        remaining_loss = self.__sig.handle_breach(remain_collat_loss)
        
        frac = self.__df_accounts.get_amount(self.__cm_index) / self.__df_accounts.total_default_fund()
        
        partial_loss = frac*remaining_loss
        
        tmp_cva = self.__discount(self.__tau_delta) * partial_loss
        self.results["cva"] = tmp_cva

In [24]:
class FillVMandIMTask(ITask):
    level = 0
    
    def __init__(self, **kwargs):
        self._delta = kwargs["delta"]
        
        self._port = kwargs['portfolio']        
        self._vm_accounts = kwargs['vm_accounts']
        self._im_accounts = kwargs['im_accounts']
        
        self._quantile_im = kwargs['quantile_im']
        
        self._zeros = np.zeros(self._port.derivatives.size)
        self._zeros.fill(1e-5)
        
    def perform(self, **kwargs):
        self._tau_minus = kwargs["t"]
        
        #############
        ## VM part ##
        #############        
        contract_prices = [d.price(self._tau_minus) for d in self._port.derivatives]
        p_tau_minus = self._port.compute_value(contract_prices)
        
        for ii, p_and_l in enumerate(p_tau_minus):
            self._vm_accounts.put_amounts(ii, p_and_l)
        
        #############
        ## IM part ##
        #############        
        exposures = self._port.compute_exposure(self._tau_minus, risk_period=self._delta, conf_level=self._quantile_im)
        for ii, ee in enumerate(exposures):
            self._im_accounts.put_amounts(ii, np.maximum(ee, self._zeros).tolist())

In [25]:
class FillIMVMandDFTask(FillVMandIMTask):
    def __init__(self, **kwargs):
        super(FillIMVMandDFTask, self).__init__(**kwargs)
        self._df_accounts = kwargs['df_accounts']        
        self._prop_df = kwargs['prop_df']
        
    def perform(self, **kwargs):
        super(FillIMVMandDFTask, self).perform(**kwargs)
        self._t = kwargs["t"]
        
        exposures = self._im_accounts.amounts*self._prop_df
        for ii, ee in enumerate(exposures):
            self._df_accounts.put_amounts(ii, ee)

In [26]:
class ReFillSigTask(ITask):
    level = 0
    
    def __init__(self, **kwargs):
        self.__sig = kwargs['sig']
        
        self.__regul_cap = kwargs['regul_capital']
        self.__risk_period = kwargs['cap_risk_period']
        self.__risk_horizon = kwargs['cap_risk_horizon']
        self.__conf_level = kwargs['cap_conf_level']
        
        self.__collat_task = FillIMVMandDFTask(delta=kwargs['delta'], portfolio=kwargs['portfolio'],
                                               vm_accounts=kwargs['vm_accounts'],
                                               im_accounts=kwargs['im_accounts'], 
                                               quantile_im=kwargs['quantile_im'],
                                               df_accounts=kwargs['df_accounts'], 
                                               prop_df=kwargs['prop_df'])
    
    def perform(self, **kwargs):
        t = kwargs["t"]        
        
        self.__collat_task.perform(t=t)
        
        if t == 0.:
            self.__sig.recover()
        else:
            self.__sig.update_value(t, 
                                    regul_capital=self.__regul_cap, 
                                    risk_horizon=self.__risk_horizon, 
                                    risk_period=self.__risk_period,
                                    conf_level=self.__conf_level)

In [27]:
from risk.basel.exposures import SABaselExposure

class ComputeDFKVATask(ITask):
    level = 1
    
    def __init__(self, **kwargs):        
        self.__cm_index = kwargs["bank_index"]
        
        self.__delta = kwargs["delta"]
        self.__exp_distrib = kwargs["exp_distrib"]
        
        self.__copula = kwargs["copula"]
        self.__cm_subset_indexes = kwargs["bank_subsets_indexes"]
        
        self.__port = kwargs['portfolio']        
        self.__time_grid = self.__port.derivatives[0].underlying.time
        
        self.__vm_accounts = kwargs['vm_accounts']
        self.__im_accounts = kwargs['im_accounts']
        self.__df_accounts = kwargs['df_accounts']
        
        self.__discount = kwargs["discount"]
        
        self.__regul_cap = kwargs['regul_capital']
        self.__risk_period = kwargs['cap_risk_period']
        self.__risk_horizon = kwargs['cap_risk_horizon']
        self.__conf_level95 = kwargs['cap_conf_level95']
        self.__conf_level99 = kwargs['cap_conf_level99']
        
        self.__c_zeta = kwargs["c_zeta"]
        self.__cm_recov = kwargs["bank_recov"]
        self.__lambda_zeta = kwargs["lambda_zeta"]
        self.__k_zeta = kwargs["k_zeta"]
        
        self.__collat_task = FillIMVMandDFTask(delta=delta, portfolio=self.__port,
                                               vm_accounts=self.__vm_accounts,
                                               im_accounts=self.__im_accounts, 
                                               quantile_im=kwargs['quantile_im'],
                                               df_accounts=self.__df_accounts, 
                                               prop_df=kwargs['prop_df'])
        
        self.__k_exposure_sa = SABaselExposure(self.__port)
        
        self.results = {}
    
    def perform(self, **kwargs):
        self.__zeta = kwargs["t"]
        self.__zeta_delta = time_offseter(self.__zeta+self.__delta, self.__time_grid, True) 
        
        inv_pdf = 1./self.__exp_distrib.pdf(zeta)
        
        beta_zeta = self.__discount(self.__zeta)
        
        self.__collat_task.perform(t=self.__zeta)
        
        ################
        ### DVA part ###
        ################        
        contract_prices = [d.price(self.__zeta_delta) for d in self.__port.derivatives]
        p_zeta_delta = port.compute_value(contract_prices)
        q_zeta_delta = p_zeta_delta
        collateral = self.__vm_accounts.amounts + self.__im_accounts.amounts + self.__df_accounts.amounts        
        
        q_zeta_delta = q_zeta_delta[self.__cm_index].sum()
        collateral = collateral[self.__cm_index].sum()
        
        loss = np.maximum(q_zeta_delta - collateral, 0.)
        
        gamma_zeta = self.__copula.tot_gamma(self.__zeta, subsets_indexes=self.__cm_subset_indexes)
        
        self.results["dva"] = inv_pdf*self.__discount(self.__zeta_delta)*gamma_zeta*loss
        
        ################
        ### FVA part ###
        ################
        contract_prices = [d.price(self.__zeta) for d in self.__port.derivatives]
        p_zeta = port.compute_value(contract_prices)
        fund = p_zeta[self.__cm_index].sum() - collateral
        
        bar_lambda_zeta = gamma_zeta*0.6
        
        tmp_fva = -self.__c_zeta * collateral
        tmp_fva += -bar_lambda_zeta * np.minimum(fund, 0.)
        tmp_fva += -self.__lambda_zeta * np.maximum(fund, 0.)
        
        self.results["fva"] = inv_pdf*beta_zeta*tmp_fva
        
        ################
        ### KVA part ###
        ################
        kva95 = self.__regul_cap.compute_kcm(self.__cm_index, self.__zeta,
                                             risk_horizon=self.__risk_horizon, 
                                             risk_period=self.__risk_period,
                                             conf_level=self.__conf_level95)
        
        kva99 = self.__regul_cap.compute_kcm(self.__cm_index, self.__zeta,
                                             risk_horizon=self.__risk_horizon, 
                                             risk_period=self.__risk_period,
                                             conf_level=self.__conf_level99)
            
        pf_losses = self.__k_exposure_sa(t=self.__zeta)
        kvasa = self.__regul_cap.compute_kcm(self.__cm_index, self.__zeta,
                                             pf_losses=pf_losses)
        
        self.results["kva95"] = inv_pdf*self.__k_zeta*beta_zeta*kva95
        self.results["kva99"] = inv_pdf*self.__k_zeta*beta_zeta*kva99
        self.results["kvasa"] = inv_pdf*self.__k_zeta*beta_zeta*kvasa

In [28]:
def merge_dict(actual_dict, to_add):
    for k, v in to_add.iteritems():
        if k not in actual_dict:
            actual_dict[k] = []
            
        actual_dict[k].append(v)
        
def sort_tasks_dict(a_dict):
    for k, v in a_dict.iteritems():
        v.sort(key=lambda x: x.level)

#MC loop

In [29]:
from utils import time_offseter

In [30]:
N = loaded_data["N"]

In [31]:
results_globs = {"cva": {"sum": 0., "sum2": 0.},
                 "dva": {"sum": 0., "sum2": 0.},
                 "fva": {"sum": 0., "sum2": 0.},
                 "kva95": {"sum": 0., "sum2": 0.},
                 "kva99": {"sum": 0., "sum2": 0.},
                 "kvasa": {"sum": 0., "sum2": 0.}}

In [32]:
from scipy.stats import expon

exp_distrib_params = loaded_data["randomization"]["distrib"]
exp_distrib = expon(loc=exp_distrib_params["loc"], scale=exp_distrib_params["scale"])

zetas = loaded_data["randomization"]["zetas"]
default_times_mat = loaded_data["credit"]["default_times"]

In [33]:
sig_time_grid = [t for t in time_grid if t.is_integer()]
df_time_grid = [t for t in time_grid if (12*t).is_integer()]

In [34]:
import time

tic = time.time()

b_df_times_indexes = [ii for ii, ind in enumerate(bc_subsets_indexes) if ind in b_subsets_indexes]

for i in range(N):
    # Resurrecting the states
    # resets to 0. the values
    # of the VMs and IMs
    states.resurrect_all()
    
    zeta = zetas[i]
    
    # We set the simulated underlying here
    # that has the all time grid with
    # mpor = 1./360
    swap.underlying = udlyings[i]
    
    default_times = default_times_mat[i]
    b_min_df_time = default_times[b_df_times_indexes].min()
    bar_tau = min(b_min_df_time, maturity)
    
    df_times_dict = {
        t: PutMembersAtDefaultTask(states=states, defaulters=copula.subsets[bc_subsets_indexes[j]])
        for j, t in enumerate(default_times) if t < bar_tau
    }
    
    liq_times_dict = {
        time_offseter(t+delta, time_grid): 
        LiquidatePortfolioTask(bank_index=b_id, default_task=task, 
                               portfolio=port, discount=discount, sig=sig,
                               vm_accounts=vm_accounts, im_accounts=im_accounts, 
                               df_accounts=df_accounts)
        for t, task in df_times_dict.iteritems() if t+delta < bar_tau        
    }
    
    tau_minus_times_dict = {
        time_offseter(t-delta, time_grid, True):
        FillVMandIMTask(delta=delta, portfolio=port, 
                        vm_accounts=vm_accounts, im_accounts=im_accounts,
                        quantile_im=quantile_im)        
        for t in df_times_dict.keys()
    }
    
    dfund_times = [time_offseter(t, df_time_grid, True) for t in df_times_dict.keys()]    
    dfund_times_dict = {
        t: FillIMVMandDFTask(delta=delta, portfolio=port, 
                             vm_accounts=vm_accounts, 
                             im_accounts=im_accounts, quantile_im=quantile_im,
                             df_accounts=df_accounts, prop_df=prop_df)
        for t in dfund_times
    }

    sig_times = {
        t: ReFillSigTask(delta=delta, portfolio=port, 
                         vm_accounts=vm_accounts, 
                         im_accounts=im_accounts, quantile_im=quantile_im,
                         df_accounts=df_accounts, prop_df=prop_df,
                         sig=sig, regul_capital=regul_cap, 
                         cap_risk_horizon=cap_risk_horizon, 
                         cap_risk_period=cap_risk_period,
                         cap_conf_level=cap_conf_level_95)
        for t in sig_time_grid if t < zeta
    }

    dfkva_time = {
        t: ComputeDFKVATask(bank_index=b_id, exp_distrib=exp_distrib, 
                            copula=copula, portfolio=port, delta=delta,
                            bank_subsets_indexes=b_subsets_indexes,
                            vm_accounts=vm_accounts, 
                            im_accounts=im_accounts, quantile_im=quantile_im,
                            df_accounts=df_accounts, prop_df=prop_df,
                            discount=discount, 
                            regul_capital=regul_cap, 
                            cap_risk_horizon=cap_risk_horizon, 
                            cap_risk_period=cap_risk_period,
                            cap_conf_level95=cap_conf_level_95, 
                            cap_conf_level99=cap_conf_level_99, 
                            c_zeta=c_0, 
                            lambda_zeta=lambda_, k_zeta=k_, bank_recov=bank_recov)
        for t in [zeta] if zeta <= bar_tau
    }
    
    current_times_dict = dict()
    all_dicts = [df_times_dict, liq_times_dict, tau_minus_times_dict, \
                 dfund_times_dict, sig_times, dfkva_time]
    
    for dd in all_dicts:
        merge_dict(current_times_dict, dd)
    
    sort_tasks_dict(current_times_dict)
    
    current_times = current_times_dict.keys()
    current_times.sort()
    
    for t in current_times:
        tasks = current_times_dict[t]
        for task in tasks:
            task.perform(t=t)
        
    cva_, dva_, fva_, kva95_, kva99_, kvasa_ = [0. for _ in range(6)]
    for k, v in liq_times_dict.iteritems():
        cva_ += v.results["cva"]
        
    results_globs["cva"]["sum"] += cva_
    results_globs["cva"]["sum2"] += cva_**2
    
    for k, v in dfkva_time.iteritems():
        dva_ += v.results["dva"]
        fva_ += v.results["fva"]
        kva95_ += v.results["kva95"]
        kva99_ += v.results["kva99"]
        kvasa_ += v.results["kvasa"]
        
    results_globs["dva"]["sum"] += dva_
    results_globs["dva"]["sum2"] += dva_**2
    
    results_globs["fva"]["sum"] += fva_
    results_globs["fva"]["sum2"] += fva_**2
    
    results_globs["kva95"]["sum"] += kva95_
    results_globs["kva95"]["sum2"] += kva95_**2
    
    results_globs["kva99"]["sum"] += kva99_
    results_globs["kva99"]["sum2"] += kva99_**2
    
    results_globs["kvasa"]["sum"] += kvasa_
    results_globs["kvasa"]["sum2"] += kvasa_**2
    
toc = time.time()

#Results

In [35]:
print "Results for %d iterations (%s secs.)"%(N, toc-tic)
print

print "Used discount factor: %s"%discount
print

print "Bank index %i that belongs to the following MO copula subsets:"%b_id
for idx in b_subsets_indexes:
    print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[idx][0]], \
                                                     copula.pillars[idx], copula.intensities[idx])
    
print "Counterparties indexes:"
print c_ids

print "\n-----------------------------------------------------\n"

for idx in c_ids:
    print "Counterparty index %s belongs to the following MO copula subses:"%idx
    sub_indexes = copula.get_indexes_including(idx)
    for subset_idx in sub_indexes:
        print "- %s with pillars %s and intensity %s\n"%([x for x in copula.subsets[subset_idx][0]], \
                                                         copula.pillars[subset_idx], copula.intensities[subset_idx])        
    print
    
print "\n-----------------------------------------------------\n"
    
print "Derivatives:"
for d in [swap]:
    print "- %s"%d
    
print "\nPositions:"
print port.positions

print "\nConfidence level used for IM: %.2f"%quantile_im
print "Proportion of IM used for DF: %.3f"%prop_df

Results for 20000 iterations (2242.602 secs.)

Used discount factor: Constant discount factor process with rate r = 0.02

Bank index 64 that belongs to the following MO copula subsets:
- [64] with pillars [3 5] and intensity [ 0.00795383  0.01045393]

- [64, 98, 99, 38, 70, 71, 104, 75, 76, 81, 78, 111, 17, 83, 120, 26, 124, 29, 31] with pillars [3 5] and intensity [ 0.0053745  0.0050889]

- [17, 26, 29, 31, 38, 50, 55, 64, 66, 70, 71, 75, 76, 78, 81, 83, 98, 99, 103, 104, 107, 111, 120, 122, 124] with pillars [3 5] and intensity [ 0.0006145  0.0086692]

- [0, 3, 4, 5, 9, 13, 17, 24, 25, 26, 27, 28, 29, 30, 31, 33, 36, 37, 38, 43, 45, 47, 50, 52, 55, 56, 57, 61, 64, 66, 70, 71, 72, 73, 75, 76, 78, 80, 81, 82, 83, 86, 87, 89, 90, 98, 99, 100, 103, 104, 107, 111, 112, 113, 114, 116, 118, 119, 120, 122, 124] with pillars [3 5] and intensity [  2.01650000e-04   9.33720000e-05]

- [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 

In [36]:
from scipy.stats import norm

conf_level = 0.95
z_level = norm.ppf(0.5*(1+conf_level))

print "Results of the global xVA:\n"
keys = results_globs.keys()
keys.sort()

excel_results = dict()

for k in keys:
    mean_ = results_globs[k]['sum']/N
    mod_var_ = (results_globs[k]['sum2']/N - mean_**2) / (N-1.)
    half_inter = z_level*np.sqrt(mod_var_)
    
    excel_results[k.upper()] = "%s \t %s"%(mean_, half_inter / mean_)
    
    print "The %s for the bank lies in [%f, %f]"%(k.upper(), mean_-half_inter, mean_+half_inter)

Results of the global xVA:

The CVA for the bank lies in [0.024748, 0.033062]
The DVA for the bank lies in [0.012292, 0.016118]
The FVA for the bank lies in [-0.037751, -0.030625]
The KVA95 for the bank lies in [0.048283, 0.050120]
The KVA99 for the bank lies in [0.051274, 0.053139]
The KVASA for the bank lies in [0.041385, 0.043106]


In [37]:
file_ = open('../resCCP.txt', 'a')

file_.write("CCP 1vs%s - DF = %s \n"%(len(c_ids), prop_df))

keys = excel_results.keys()
keys.sort()

for k in keys:
    file_.write("%s \t %s\n"%(k, excel_results[k]))
    
file_.write("\n")
file_.close()